In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
import pathlib
url = f"/kaggle/input/jellyfish-types"

In [4]:
data = pathlib.Path(url)

In [5]:
data

PosixPath('/kaggle/input/jellyfish-types')

In [6]:
fish = {
    "Moon_jellyfish": list(data.glob('Moon_jellyfish/*.jpg')),
    "barrel_jellyfish": list(data.glob('barrel_jellyfish/*.jpg')),
    "blue_jellyfish": list(data.glob('blue_jellyfish/*.jpg')),
    "compass_jellyfish" : list(data.glob('compass_jellyfish/*.jpg')),  
    "lions_mane_jellyfish" : list(data.glob('lions_mane_jellyfish/*.jpg')),
    "mauve_stinger_jellyfish" : list(data.glob('mauve_stinger_jellyfish/*.jpg'))
}

In [7]:
labels = {
    "barrel_jellyfish" : 0,
    "blue_jellyfish":1,
    "compass_jellyfish":2,
    "lions_mane_jellyfish":3,
    "mauve_stinger_jellyfish":4,
    "Moon_jellyfish":5
}

In [8]:
X=[]
y=[]

In [9]:
import cv2

for name, images in fish.items():
    for image in images:
        try:
            image = str(image).strip()
            img = cv2.imread(image)
            if img is not None:
                img_resize = cv2.resize(img,(224,224))
                X.append(img_resize)
                y.append(labels[name]) 
            else :
                pass
        except Exception as e:
            pass

In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
X.shape

(853, 224, 224, 3)

In [12]:
y.shape

(853,)

In [13]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)

In [14]:
from sklearn.model_selection import train_test_split
X_train , X_valid , y_train , y_valid = train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")
model1 = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[224, 224, 3]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=6, activation="softmax")
])

In [16]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      9472      
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 56, 56, 256)       2

In [17]:
checkpoint_1 = tf.keras.callbacks.ModelCheckpoint("checkpoints_1",save_weights_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
model1.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model1.fit(X_train, y_train, epochs=50,
                    validation_data=(X_valid, y_valid),callbacks=[checkpoint_1,early_stopping])

Epoch 1/50
22/22 [==============================] - 27s 398ms/step - loss: 572.9877 - accuracy: 0.1466 - val_loss: 1.7672 - val_accuracy: 0.1988
Epoch 2/50
22/22 [==============================] - 5s 236ms/step - loss: 1.7944 - accuracy: 0.1745 - val_loss: 1.7652 - val_accuracy: 0.2573
Epoch 3/50
22/22 [==============================] - 6s 254ms/step - loss: 1.7647 - accuracy: 0.2053 - val_loss: 1.7230 - val_accuracy: 0.1754
Epoch 4/50
22/22 [==============================] - 5s 236ms/step - loss: 1.7619 - accuracy: 0.2067 - val_loss: 1.7211 - val_accuracy: 0.2339
Epoch 5/50
22/22 [==============================] - 6s 269ms/step - loss: 1.7582 - accuracy: 0.2126 - val_loss: 1.7274 - val_accuracy: 0.2456
Epoch 6/50
22/22 [==============================] - 5s 251ms/step - loss: 1.7569 - accuracy: 0.2097 - val_loss: 1.6964 - val_accuracy: 0.2339
Epoch 7/50
22/22 [==============================] - 6s 274ms/step - loss: 1.7357 - accuracy: 0.1891 - val_loss: 1.6946 - val_accuracy: 0.2222
Epo

In [18]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [19]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

model2 = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
])
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model2.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model2.add(tf.keras.layers.GlobalAvgPool2D())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(6, activation="softmax"))

In [20]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 112, 112, 64)      9408      
                                                                 
 batch_normalization (Batch  (None, 112, 112, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)        0         
 D)                                                              
                                                                 
 residual_unit (ResidualUni  (None, 56, 56, 64)        74240     
 t)                                                              
                                                        

In [21]:
checkpoint_2 = tf.keras.callbacks.ModelCheckpoint("checkpoints_2",save_weights_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
model2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model2.fit(X_train, y_train, epochs=50,
                    validation_data=(X_valid, y_valid),callbacks=[checkpoint_2,early_stopping])

Epoch 1/50
22/22 [==============================] - 39s 335ms/step - loss: 2.6353 - accuracy: 0.2786 - val_loss: 170372.2656 - val_accuracy: 0.1696
Epoch 2/50
22/22 [==============================] - 5s 238ms/step - loss: 1.4855 - accuracy: 0.4047 - val_loss: 15980.4297 - val_accuracy: 0.1696
Epoch 3/50
22/22 [==============================] - 5s 248ms/step - loss: 1.2837 - accuracy: 0.4956 - val_loss: 3005.9675 - val_accuracy: 0.2398
Epoch 4/50
22/22 [==============================] - 5s 237ms/step - loss: 1.0551 - accuracy: 0.5704 - val_loss: 60.0301 - val_accuracy: 0.1520
Epoch 5/50
22/22 [==============================] - 6s 269ms/step - loss: 1.0106 - accuracy: 0.6364 - val_loss: 53.1591 - val_accuracy: 0.1754
Epoch 6/50
22/22 [==============================] - 5s 242ms/step - loss: 0.9129 - accuracy: 0.6833 - val_loss: 36.5085 - val_accuracy: 0.3099
Epoch 7/50
22/22 [==============================] - 6s 254ms/step - loss: 0.8430 - accuracy: 0.6745 - val_loss: 30.4558 - val_accura

In [22]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

model3 = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224,224, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
])

residual_blocks = [3, 4, 6, 3]  # ResNet-50 configuration

prev_filters = 64
for filters, blocks in zip([64, 128, 256, 512], residual_blocks):
    for _ in range(blocks):
        if filters != prev_filters:
            strides = 2
        else:
            strides = 1
        model3.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters

model3.add(tf.keras.layers.GlobalAveragePooling2D())
model3.add(tf.keras.layers.Dense(6, activation="softmax"))

In [23]:
checkpoint_3 = tf.keras.callbacks.ModelCheckpoint("checkpoints_3",save_weights_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
model3.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model3.fit(X_train, y_train, epochs=50,
                    validation_data=(X_valid, y_valid),callbacks=[checkpoint_3,early_stopping])

Epoch 1/50
22/22 [==============================] - 30s 272ms/step - loss: 3.1450 - accuracy: 0.2551 - val_loss: 432993.2500 - val_accuracy: 0.1404
Epoch 2/50
22/22 [==============================] - 5s 235ms/step - loss: 1.4496 - accuracy: 0.4282 - val_loss: 28092.6250 - val_accuracy: 0.1404
Epoch 3/50
22/22 [==============================] - 5s 238ms/step - loss: 1.2011 - accuracy: 0.5220 - val_loss: 2585.9421 - val_accuracy: 0.1404
Epoch 4/50
22/22 [==============================] - 6s 269ms/step - loss: 1.0267 - accuracy: 0.6085 - val_loss: 134.3838 - val_accuracy: 0.1404
Epoch 5/50
22/22 [==============================] - 5s 239ms/step - loss: 0.9452 - accuracy: 0.6716 - val_loss: 70.1450 - val_accuracy: 0.1345
Epoch 6/50
22/22 [==============================] - 6s 273ms/step - loss: 0.8339 - accuracy: 0.6877 - val_loss: 20.0165 - val_accuracy: 0.1754
Epoch 7/50
22/22 [==============================] - 5s 231ms/step - loss: 0.8040 - accuracy: 0.6686 - val_loss: 37.5583 - val_accur